In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#!pip install transformers -qq
# !pip install sentencepiece -qq
# !pip install tokenizer -qq
#!pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 5.5 MB/s eta 0:00:00


## Set Cuda

In [ ]:
import torch
from torchtext import data
from torchtext import datasets
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

torch.cuda.is_available()

True

##Load Dataset

In [ ]:
from sklearn.model_selection import train_test_split

def to_df(x, y):
    d = {"text": x, "label": y}
    return pd.DataFrame(d)

def split_3(df, test_size=0.2, valid_size=0.2):
    _df = df.copy().sample(frac=1).reset_index()
    _df = _df[["text", "label"]]

    x = df["text"].copy()
    y = df["label"].copy()
    #split train-test
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, stratify=y, random_state=SEED)
    # split train-valid
    x, y = x_train, y_train
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=valid_size, stratify=y, random_state=SEED)
    return to_df(x_train, y_train), to_df(x_valid, y_valid), to_df(x_test, y_test)

In [ ]:
import random
from itertools import chain
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
from nlpaug.util import Action


alpha_common_error = 0.10
alpha_common_error_char = 0.05
aug1_OCR = nac.OcrAug(aug_word_p=alpha_common_error)
aug2_Rins = nac.RandomCharAug(action="insert", aug_word_p=alpha_common_error, aug_char_min=1, aug_char_max=1, aug_char_p=alpha_common_error_char)
aug3_Rsub = nac.RandomCharAug(action="substitute", aug_word_p=alpha_common_error, aug_char_min=1, aug_char_max=1, aug_char_p=alpha_common_error_char)
aug4_Rswa = nac.RandomCharAug(action="swap", aug_word_p=alpha_common_error,aug_char_min=1, aug_char_max=1, aug_char_p=alpha_common_error_char) #
aug5_Rdel = nac.RandomCharAug(action="delete", aug_word_p=alpha_common_error, aug_char_min=1, aug_char_max=1, aug_char_p=alpha_common_error_char)
aug6_Kb = nac.KeyboardAug(aug_word_p=alpha_common_error)
aug7_Split = naw.SplitAug(aug_p=alpha_common_error)


def text2augment(text, m):
    output = [text, ]

    temp = random.sample(range(0, 7), m - 1)

    if 0 in temp:
        output.append( *aug1_OCR.augment(text))
    if 1 in temp:
        output.append( *aug2_Rins.augment(text))
    if 2 in temp:
        output.append( *aug3_Rsub.augment(text))
    if 3 in temp:
        output.append( *aug4_Rswa.augment(text))
    if 4 in temp:
        output.append( *aug5_Rdel.augment(text))
    if 5 in temp:
        output.append( *aug6_Kb.augment(text))
    if 6 in temp:
        output.append( *aug7_Split.augment(text))

    return output


def aug_replicate(y_labels):
    return list(chain(* [[y]*(1 if y == 0 else 2) for y in y_labels] ))

def aug_text(x_text, y_labels):
    x_text = [ text2augment(x, 1 if y == 0 else 2) for x, y in zip(x_text, y_labels)]
    return pd.Series(list(chain(*x_text)), index=None)

def split_3_aug(df, test_size=0.2, valid_size=0.2):
    _df = df.copy().sample(frac=1).reset_index()
    _df = _df[["text", "label"]]

    x = _df["text"].copy()
    y = _df["label"].copy()
    #split train-test
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, stratify=y, random_state=SEED)
    # augment
    # x_test = aug_text(x_test, y_test)
    # y_test = aug_replicate(y_test)
    # split train-valid
    x, y = x_train, y_train
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=valid_size, stratify=y, random_state=SEED)
    # augment
    x_train = aug_text(x_train, y_train)
    y_train = aug_replicate(y_train)
    x_valid = aug_text(x_valid, y_valid)
    y_valid = aug_replicate(y_valid)

    print(x_valid.shape)
    print("DONE")
    print(len(y_valid))

    print(x_train.shape)
    print("DONE")
    print(len(y_train))

    return to_df(x_train, y_train), to_df(x_valid, y_valid), to_df(x_test, y_test)

In [ ]:
import numpy as np
import pandas as pd

tname_data = "./hsd_merge_cleaned_lowered"
data = pd.read_csv(f"{tname_data}.csv")

train, valid, test = split_3_aug(data)

X_train = train['text']
y_train = train['label']

X_valid = valid['text']
y_valid = valid['label']

X_test = test['text']
y_test = test['label']

# Extract feature by using BETO

In [ ]:
import pandas as pd
from glob import glob

train_sentences = list(train['text'].values)
train_labels = list(train['label'].values)

valid_sentences = list(valid['text'].values)
valid_labels = list(valid['label'].values)

test_sentences = list(test['text'].values)
test_labels = list(test['label'].values)

Load tokenizer of BETO

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

In [ ]:
#choose max_length for phobert model based on the input length

max_length = 0
list_len=[]
for sentence in train_sentences:
    length = len(tokenizer.tokenize(sentence))
    list_len.append(length)

from collections import Counter
Counter(list_len).most_common(100)

[(17, 404),
 (15, 380),
 (23, 354),
 (18, 344),
 (24, 343),
 (19, 340),
 (22, 337),
 (21, 336),
 (20, 335),
 (14, 328),
 (12, 325),
 (13, 325),
 (16, 325),
 (26, 310),
 (9, 302),
 (11, 298),
 (10, 293),
 (25, 277),
 (29, 271),
 (27, 270),
 (28, 262),
 (31, 210),
 (7, 210),
 (32, 201),
 (30, 200),
 (8, 192),
 (33, 189),
 (34, 148),
 (6, 144),
 (36, 137),
 (38, 121),
 (53, 117),
 (39, 117),
 (35, 117),
 (37, 117),
 (40, 116),
 (5, 115),
 (42, 112),
 (50, 111),
 (43, 109),
 (54, 109),
 (41, 106),
 (44, 106),
 (49, 105),
 (45, 102),
 (48, 101),
 (52, 100),
 (56, 98),
 (51, 98),
 (46, 97),
 (47, 92),
 (55, 85),
 (58, 85),
 (57, 84),
 (4, 67),
 (59, 61),
 (60, 58),
 (61, 55),
 (64, 38),
 (62, 36),
 (65, 33),
 (3, 31),
 (66, 28),
 (63, 26),
 (67, 17),
 (69, 13),
 (68, 12),
 (2, 7),
 (72, 6),
 (70, 6),
 (1, 4),
 (71, 2),
 (73, 1),
 (74, 1),
 (81, 1)]

In [ ]:
# Encode train label

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(train_labels)
encoded_labels = le.transform(train_labels)
encoded_test_labels = le.transform(valid_labels)

In [ ]:
# Tokens IDs tensor

def encoder_generator(sentences,labels):

    sent_index = []
    input_ids = []
    attention_masks =[]

    for index,sent in enumerate(sentences):

        sent_index.append(index)

        encoded_dict = tokenizer.encode_plus(sent,
                                             add_special_tokens=True,
                                             max_length=50,
                                             pad_to_max_length=True,
                                             truncation = True,
                                             return_attention_mask=True,
                                             return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])

        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids,dim=0).cuda()
    attention_masks = torch.cat(attention_masks,dim=0).cuda()
    labels = torch.tensor(labels).cuda()
    sent_index = torch.tensor(sent_index).cuda()

    return sent_index,input_ids,attention_masks,labels

train_sent_index,train_input_ids,train_attention_masks,train_encoded_label_tensors = encoder_generator(train_sentences,encoded_labels)
valid_sent_index,valid_input_ids,valid_attention_masks,valid_encoded_label_tensors = encoder_generator(valid_sentences,encoded_test_labels)
print('Original: ', train_sentences[0])
print('Token IDs:', train_input_ids[0])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  maripan ctm siempre mandandose cagas
Token IDs: tensor([    4,  4352,  5249,  1016, 30940, 30943,  2032,  4521, 30935,  2610,
         1285,  1723,     5,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
       device='cuda:0')


In [ ]:
# Connvert train, dev input by using TensorDataset

from torch.utils.data import TensorDataset,random_split

train_dataset = TensorDataset(train_input_ids,train_attention_masks,train_encoded_label_tensors)
valid_dataset = TensorDataset(valid_input_ids,valid_attention_masks,valid_encoded_label_tensors)

print('train data samples is {}'.format(len(train_dataset)))
print("valid data samples is {}".format(len(valid_dataset)))

train data samples is 11413
valid data samples is 2854


In [ ]:
# Set cuda by using device

from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

bs=128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_data_loader = DataLoader(train_dataset,
                              sampler=RandomSampler(train_dataset),
                              batch_size=bs)
valid_data_loader = DataLoader(valid_dataset,
                              sampler=RandomSampler(valid_dataset),
                              batch_size=bs)

Load model BETO

In [ ]:
from transformers import AutoModel

beto = AutoModel.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')
beto = beto.to(device)

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Build LSTM

In [ ]:
import torch
import torch.nn as nn

import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim,
                            hidden_dim,
                            num_layers = n_layers,
                            # bidirectional = True,
                            dropout = 0.1,
                            batch_first = True
                           )

        self.fc = nn.Linear(hidden_dim * 2,output_dim)

    def forward(self, x, x_lengths):
        embedded = self.embedding(x)

        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, torch.sum(x_lengths, dim=1).cpu(), batch_first=True, enforce_sorted=False)

        packed_output,(hidden_state,cell_state) = self.lstm(packed_embedded)

        hidden = torch.cat((hidden_state[-2,:,:], hidden_state[-1,:,:]), dim = 1)

        logits=self.fc(hidden)
        return logits


In [ ]:
# Definir los parámetros del modelo
vocab_size = 31002#tamaño del vocabulario
embedding_dim = 768 #Dimension de los vectores de embedding
hidden_size = 128  # Tamaño de la capa oculta del LSTM
num_layers = 2  # Número de capas del LSTM
output_size = 2  # Tamaño de la salida

# Crear una instancia del modelo LSTM
LSTMmodel = LSTMModel(vocab_size, embedding_dim, hidden_size, output_size, num_layers)

In [ ]:

# Imprimir el modelo
print(LSTMmodel)

LSTMModel(
  (embedding): Embedding(31002, 768)
  (lstm): LSTM(768, 128, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (fc): Linear(in_features=256, out_features=2, bias=True)
)


In [ ]:
# Optimizer and criterion

import torch.optim as optim

model_parameters = list(LSTMmodel.parameters())

optimizer = optim.Adam(model_parameters,lr=2e-5,eps=1e-8)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

In [ ]:
# Calculate accuracy per batch during train

def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]]).cuda()

In [ ]:
# Def for training

from tqdm import tqdm

def train(model):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in tqdm(train_data_loader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        optimizer.zero_grad()

        predictions = model(b_input_ids,b_input_mask)

        loss = criterion(predictions, b_labels)

        acc = categorical_accuracy(predictions, b_labels)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(train_data_loader), epoch_acc / len(train_data_loader)

In [ ]:
# Class for predict label

import numpy as np

def predictions_labels(preds,labels):
    pred = np.argmax(preds,axis=1).flatten()
    label = labels.flatten()
    return pred,label

In [ ]:
# Evaluate loss, acc  and f1-macro

from sklearn.metrics import classification_report,accuracy_score,f1_score
def eval(model):
    epoch_loss = 0

    total_predictions = []
    total_true = []

    all_true_labels = []
    all_pred_labels = []

    model.eval()

    with torch.no_grad():

        for batch in tqdm(valid_data_loader):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            predictions = model(b_input_ids,b_input_mask)

            loss = criterion(predictions, b_labels)
            epoch_loss += loss.item()

            predictions = predictions.detach().cpu().numpy()

            label_ids = b_labels.to('cpu').numpy()

            pred,true = predictions_labels(predictions,label_ids)

            all_pred_labels.extend(pred)
            all_true_labels.extend(true)

    print(classification_report(all_pred_labels,all_true_labels))
    avg_val_accuracy = accuracy_score(all_pred_labels,all_true_labels)
    macro_f1_score = f1_score(all_pred_labels,all_true_labels,average='macro')

    avg_val_loss = epoch_loss/len(valid_data_loader)

    print("accuracy = {0:.2f}".format(avg_val_accuracy))

    return avg_val_loss,avg_val_accuracy,macro_f1_score

In [ ]:
# Time for training

import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# Set device and gpu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

LSTMmodel.cuda()

LSTMModel(
  (embedding): Embedding(31002, 768)
  (lstm): LSTM(768, 128, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (fc): Linear(in_features=256, out_features=2, bias=True)
)

# Training

In [ ]:
tempname = './lstm_aug_model'

In [ ]:
epochs = 20

best_macro_f1 = float('0')

for epoch in range(epochs):

    start_time = time.time()
    train_loss,train_acc = train(LSTMmodel)
    valid_loss,valid_acc,macro_f1 = eval(LSTMmodel)
    end_time = time.time()


    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
    torch.save(LSTMmodel, tempname+'_task2a_2.pt')
    print("model saved")

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. acc: {valid_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. F1: {macro_f1*100:.2f}%')
    print('=============Epoch Ended==============')

100%|██████████| 23/23 [00:00<00:00, 124.41it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      0.82      0.90      2854
           1       0.00      0.00      0.00         0

    accuracy                           0.82      2854
   macro avg       0.50      0.41      0.45      2854
weighted avg       1.00      0.82      0.90      2854

accuracy = 0.82
model saved
Epoch: 01 | Epoch Time: 0m 2s
	Train Loss: 0.575 | Train acc: 81.09%
	 Val. Loss: 0.490 |  Val. acc: 82.20%
	 Val. Loss: 0.490 |  Val. F1: 45.12%
=============Epoch Ended==============


100%|██████████| 23/23 [00:00<00:00, 169.55it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      0.82      0.90      2854
           1       0.00      0.00      0.00         0

    accuracy                           0.82      2854
   macro avg       0.50      0.41      0.45      2854
weighted avg       1.00      0.82      0.90      2854

accuracy = 0.82
model saved
Epoch: 02 | Epoch Time: 0m 2s
	Train Loss: 0.471 | Train acc: 82.29%
	 Val. Loss: 0.470 |  Val. acc: 82.20%
	 Val. Loss: 0.470 |  Val. F1: 45.12%
=============Epoch Ended==============


100%|██████████| 23/23 [00:00<00:00, 166.89it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

              precision    recall  f1-score   support

           0       0.94      0.88      0.91      2509
           1       0.41      0.60      0.48       345

    accuracy                           0.85      2854
   macro avg       0.67      0.74      0.70      2854
weighted avg       0.88      0.85      0.86      2854

accuracy = 0.85
model saved
Epoch: 18 | Epoch Time: 0m 2s
	Train Loss: 0.207 | Train acc: 91.53%
	 Val. Loss: 0.402 |  Val. acc: 84.55%
	 Val. Loss: 0.402 |  Val. F1: 69.61%
=============Epoch Ended==============


100%|██████████| 23/23 [00:00<00:00, 159.20it/s]


              precision    recall  f1-score   support

           0       0.93      0.88      0.91      2474
           1       0.44      0.58      0.50       380

    accuracy                           0.84      2854
   macro avg       0.68      0.73      0.70      2854
weighted avg       0.87      0.84      0.85      2854

accuracy = 0.84
model saved
Epoch: 19 | Epoch Time: 0m 2s
	Train Loss: 0.187 | Train acc: 92.40%
	 Val. Loss: 0.413 |  Val. acc: 84.37%
	 Val. Loss: 0.413 |  Val. F1: 70.26%
=============Epoch Ended==============


100%|██████████| 23/23 [00:00<00:00, 166.33it/s]


              precision    recall  f1-score   support

           0       0.94      0.88      0.91      2512
           1       0.41      0.61      0.49       342

    accuracy                           0.85      2854
   macro avg       0.67      0.74      0.70      2854
weighted avg       0.88      0.85      0.86      2854

accuracy = 0.85
model saved
Epoch: 20 | Epoch Time: 0m 2s
	Train Loss: 0.172 | Train acc: 93.35%
	 Val. Loss: 0.443 |  Val. acc: 84.72%
	 Val. Loss: 0.443 |  Val. F1: 69.87%
=============Epoch Ended==============


In [ ]:
# Save BETO and CNN

torch.save(LSTMmodel,tempname + 'module2_part1.pt')


## EVALUATING

In [ ]:
# Load phobert and cnn

import torch
LSTMmodel = torch.load(tempname + '_task2a_2.pt')
LSTMmodel.eval()



LSTMModel(
  (embedding): Embedding(31002, 768)
  (lstm): LSTM(768, 128, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (fc): Linear(in_features=256, out_features=2, bias=True)
)

Predict label from true label

In [ ]:
test_sent_index, test_input_ids, test_attention_masks, test_encoded_label_tensors = encoder_generator(test_sentences,test_labels)
test_dataset = TensorDataset(test_input_ids,test_attention_masks,test_encoded_label_tensors)

test_data_loader = DataLoader(test_dataset,
                              sampler=RandomSampler(test_dataset),
                              batch_size=bs)

all_pred_labels = []
all_true_labels = []

with torch.no_grad():
  for batch in tqdm(test_data_loader):
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    predictions = LSTMmodel(b_input_ids,b_input_mask)


    predictions = predictions.detach().cpu().numpy()

    label_ids = b_labels.to('cpu').numpy()

    pred, true = predictions_labels(predictions, label_ids)

    all_pred_labels.extend(pred)
    all_true_labels.extend(true)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 28/28 [00:00<00:00, 122.96it/s]


In [ ]:
# The final score in the test set (classification report)

# print(classification_report(all_pred_labels,all_true_labels, digits = 4))
print(classification_report(all_true_labels,all_pred_labels, digits = 4))

              precision    recall  f1-score   support

           0     0.8874    0.9035    0.8954      2932
           1     0.5137    0.4709    0.4914       635

    accuracy                         0.8265      3567
   macro avg     0.7006    0.6872    0.6934      3567
weighted avg     0.8209    0.8265    0.8235      3567



In [ ]:
# Confusion matrix in thetest set

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(all_true_labels, all_pred_labels)
cm

array([[2649,  283],
       [ 336,  299]])